In [ ]:
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding


raw_datasets = load_dataset('glue', 'mrpc')

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(row):
    return tokenizer(row['sentence1'], row['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [13]:
# Prepare the datasets 

tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format('torch')
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [14]:
# Define the dataloaders 

from torch.utils.data import DataLoader

batch_size = 8

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

val_dataloader = DataLoader(
    tokenized_datasets['validation'],
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [15]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 69]),
 'token_type_ids': torch.Size([8, 69]),
 'attention_mask': torch.Size([8, 69])}

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
# Test the model by passing a sample batch 

outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6920, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [18]:
# Define the optimizer and learning rate scheduler

from transformers import AdamW, get_scheduler

learning_rate = 5e-5
num_epochs = 4
num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
import torch
from tqdm.auto import tqdm


if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

progress_bar = tqdm(range(num_training_steps))

model.to(device)
model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)

        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

print('Done training.')

  0%|          | 0/1836 [00:00<?, ?it/s]

In [20]:
import evaluate


metric = evaluate.load('accuracy', 'f1')
model.eval()

for batch in val_dataloader:
    batch = {k: v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.8553921568627451}

In [21]:
import torch 

CLASSES = ['Not equivalent', 'Equivalent']

a = 'The boy quickly ran across the finish line, seizing yet another victory.' 
b = 'The quick boy seized yet another victory when he ran across the finish line.'
test_inputs = tokenizer(a, b, return_tensors='pt')
test_inputs = {k: v.to(device) for k,v in test_inputs.items()}

test_outputs = model(**test_inputs)
pred = torch.argmax(test_outputs.logits).item()
print(CLASSES[pred])

Equivalent


In [34]:
import torch.nn.functional as F

print(test_outputs.logits)
preds = F.softmax(test_outputs.logits, dim=-1)
for p in preds.squeeze():
    print(p.item())


tensor([[-3.4518,  3.5024]], device='cuda:0', grad_fn=<AddmmBackward0>)
0.0009537157602608204
0.999046266078949
